In [42]:
import numpy as np
import pandas as pd

def get_pos_list(start, n_step, step_size):
    pos_list = []
    for i in range(n_step):
        pos_list.append(round((start + i * step_size), 4))
    return pos_list

def rename_by_motor(name_list, motor_pos_list):
    assert len(name_list) == len(motor_pos_list)
    new_name_list = []
    for m, each_name in enumerate(name_list):
        _pos = str(motor_pos_list[m]).split('.')
        if len(_pos) == 1:
            new_name_list.append(each_name + "_" + _pos[0].rjust(3, '0'))
        elif len(_pos) == 2:
            new_name_list.append(each_name + "_" + _pos[0].rjust(3, '0') + "_" +  _pos[1].ljust(3,'0'))
    return new_name_list
        

In [43]:
fname = "tomo"
delay = "3"
acq_time = '30'
duplicate = 3

motor1_pos = [1.2, 5.6, 8.7]
motor2_pos = [350, 118, 25]
motor3_pos = get_pos_list(start=92.4, n_step=3, step_size=0.1)
# motor2_pos = get_pos_list(start=5, n_step=3, step_size=0.1)
# motor3_pos = get_pos_list(start=20, n_step=2, step_size=0.1)

motor_dict = {
    "CG1D:Mot:SmallRot1": motor1_pos,
#     "CG1D:Mot:LongAxis": motor2_pos,
#     "CG1D:Mot:G0:X": motor3_pos,    
}

############################################################################
num_row = 1
num_row_dict = {}
for each_motor in motor_dict.keys():
    _row = len(motor_dict[each_motor])
    num_row = num_row * _row
    num_row_dict[each_motor] = num_row
num_row_dup = num_row * 1 # duplicate
loc_to_insert_dict = {}
for each_motor in motor_dict.keys():
    loc_to_insert_dict[each_motor] = np.arange(0, num_row_dup, int(num_row_dup/num_row_dict[each_motor])).tolist()
    
###################################
output_file_name = fname+".csv"
fname_PV = "CG1D:Det:A1:TIFF1:FileName"
camera = "QHY 90"
top_dict = {fname_PV:fname,
            }
if len(motor_dict) != 0 :
    for each_motor in motor_dict.keys():
        top_dict[each_motor] = motor_dict[each_motor]
top_dict["CG1D:Det:A1:AcquireTime"] = acq_time
top_dict["Delay"] = delay
top_dict["Camera"] = camera
content_list = []
for top in top_dict.keys():
    if top in motor_dict.keys():
        content_list.append('')
    elif top == 'Delay':
        content_list.append('')
    else:
        content_list.append(top_dict[top])
scan_dict = {}
for index in range(num_row_dup):
    scan_dict[index] = content_list
scan_df = pd.DataFrame.from_dict(scan_dict, orient='index', columns=top_dict.keys())
for motor in motor_dict.keys():
    motor_dict[motor] = motor_dict[motor] * int(len(loc_to_insert_dict[motor])/len(motor_dict[motor]))
    for n, pos in enumerate(motor_dict[motor]):
        scan_df[motor][loc_to_insert_dict[motor][n]] = pos
_last_motor = list(motor_dict.keys())[-1]
for n, pos in enumerate(motor_dict[_last_motor]):
    scan_df['Delay'][loc_to_insert_dict[_last_motor][n]] = delay
scan_df[fname_PV] = rename_by_motor(scan_df[fname_PV].to_list(), scan_df["CG1D:Mot:SmallRot1"].to_list())

In [44]:
scan_df

,CG1D:Det:A1:TIFF1:FileName,CG1D:Mot:SmallRot1,CG1D:Det:A1:AcquireTime,Delay,Camera
0,tomo_001_200,1.2,30,3,QHY 90
1,tomo_005_600,5.6,30,3,QHY 90
2,tomo_008_700,8.7,30,3,QHY 90


In [23]:
scan_df.to_csv(output_file_name, index=False)